In [9]:
!pip install transformers==4.37.2 optimum==1.12.0 --quiet
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet
!pip install langchain==0.1.9 --quiet
# !pip install chromadb
!pip install sentence_transformers==2.4.0 --quiet
!pip install unstructured --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six==20221105 --quiet
!pip install unstructured-inference --quiet
!pip install faiss-gpu==1.7.2 --quiet
!pip install pikepdf==8.13.0 --quiet
!pip install pypdf==4.0.2 --quiet
!pip install pillow_heif==0.15.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.11.5 requires pdfminer.six==20231228, but you have pdfminer-six 20221105 which is incompatible.


In [10]:
import json

# Opening JSON file
f_train = open('train_TLQA.json')
f_val = open('val_TLQA.json')
f_test = open('test_TLQA.json')

# returns JSON object as a dictionary
train_data = json.load(f_train)
val_data = json.load(f_val)
test_data = json.load(f_test)

In [11]:
question = "List all sports teams Craig Bellamy, also known as Craig Douglas Bellamy, played for from 2010 to 2014."

In [12]:
import json
import os
import pandas as pd
import time
import re
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class KnnSearch:
    def __init__(self,data=None, num_trees=None,emb_dim=None):
        self.num_trees=num_trees
        self.emb_dim=emb_dim
        self.model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    def get_embeddings_for_data(self, data_ls):
        #model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        embeddings = self.model.encode(data_ls)
        return embeddings

    def get_top_n_neighbours(self, sentence, data_emb, transfer_data, k): #strategy_emb=None, str_qa=None,k):
        sent_emb = self.get_embeddings_for_data(sentence)
        #data_emb = self.get_embeddings_for_data(transfer_questions)
        top_questions = []

        print("new_emb", sent_emb.shape, data_emb.shape)
        text_sims = cosine_similarity(data_emb,[sent_emb]).tolist()
        results_sims = zip(range(len(text_sims)), text_sims)
        sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)

        #print("text_sims",sorted_similarities[:2])
        for idx, item in sorted_similarities[:k]:
                #if item[0] > 0.45:
                    top_questions.append(transfer_data[idx])

        #text_sims = cosine_similarity(strategy_emb,[sent_emb]).tolist()
        #results_sims = zip(range(len(text_sims)), text_sims)
        #sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)
        #print("text_sims",sorted_similarities[:2])
        #for idx, item in sorted_similarities:
        #        top_questions.append(str_qa[idx])
        return top_questions

In [13]:
knnSearch = KnnSearch()
questions = [item["question"] for item in train_data]

In [14]:
most_similar_questions = knnSearch.get_top_n_neighbours(sentence=question,
                                        data_emb=knnSearch.get_embeddings_for_data(questions),
                                        transfer_data=train_data,
                                        k=5)

new_emb (768,) (3212, 768)


In [15]:
print([item["question"] for item in most_similar_questions])

['List all sports teams Craig Bryson, also known as Craig James Bryson, played for from 2010 to 2020.', 'List all sports teams Craig Gordon, also known as Craig Sinclair Gordon, played for from 2010 to 2020.', 'List all sports teams Gareth Bale, also known as Gareth Frank Bale, played for from 2010 to 2020.', 'List all sports teams Mateo Kovačić, also known as Mateo Kovacic, played for from 2010 to 2020.', 'List all sports teams Landon Donovan, also known as Landon Timothy Donovan, played for from 2010 to 2018.']


In [16]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

input_text = """
Given a question, answer it accurately and format the response as a list.
Here are some examples:

Q: List all sports teams Craig Bryson, also known as Craig James Bryson, played for from 2010 to 2020.
A:
- Manchester City F.C. (2010, 2011, 2012)
- Machester United F.C. (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Q: List all sports teams Landon Donovan, also known as Landon Timothy Donovan, played for from 2010 to 2018.
A:
- P.S.V (2010, 2011)
- Feyenoord (2012, 2013, 2014, 2015, 2016, 2017, 2018)

Q: List all rugby teams Richie McCaw played for from 2001 to 2015.
A:
- Crusaders (2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015)
- New Zealand All Blacks (2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015)

Q: List all Formula 1 teams Lewis Hamilton raced for from 2007 to 2021.
A:
- McLaren (2007, 2008, 2009, 2010, 2011, 2012)
- Mercedes-AMG Petronas (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021)

Q: List all sports teams Craig Bellamy, also known as Craig Douglas Bellamy, played for from 2010 to 2014.
A:
"""
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad>- Manchester City F.C. (2010, 2011) - Manchester United F.


In [17]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Load the FlanT5-large model
flan_t5_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

# Integrate with LangChain
llm = HuggingFacePipeline(pipeline=flan_t5_pipeline)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
response = flan_t5_pipeline("""
Given a question, answer it accurately and format the response as a list.
Here are some examples:

Q: List all sports teams Craig Bryson, also known as Craig James Bryson, played for from 2010 to 2020.
A:
- Manchester City F.C. (2010, 2011, 2012)
- Machester United F.C. (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Q: List all sports teams Landon Donovan, also known as Landon Timothy Donovan, played for from 2010 to 2018.
A:
- P.S.V (2010, 2011)
- Feyenoord (2012, 2013, 2014, 2015, 2016, 2017, 2018)

Q: List all rugby teams Richie McCaw played for from 2001 to 2015.
A:
- Crusaders (2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015)
- New Zealand All Blacks (2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015)

Q: List all Formula 1 teams Lewis Hamilton raced for from 2007 to 2021.
A:
- McLaren (2007, 2008, 2009, 2010, 2011, 2012)
- Mercedes-AMG Petronas (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021)

Q: List all sports teams Craig Bellamy, also known as Craig Douglas Bellamy, played for from 2010 to 2014.
A:
""",
    max_length=200,
    temperature=0.7,
    top_p=0.9)

print(response[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'generated_text': '- Manchester City F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Newcastle United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Newcastle United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United F.C. (2010, 2011) - Manchester United'}


In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.prompts import FewShotPromptTemplate

examples = [{key: item[key] for key in ["question", "answers"]} for item in most_similar_questions]
examples_formatted = [{**item, "answers": "\n".join(f"{answer}" for answer in item["answers"])} for item in examples]
print(examples_formatted[0])


example_template = """
Question: {question}
[Answer]: {answers}
"""

example_prompt = PromptTemplate(
    input_variables=["question", "answers"],
    template=example_template,
)

#print(example_prompt.invoke(subset_data[0]).to_string())

prefix = """
You are a helpful assistant. Given a question, answer it accurately and format the response as a list.
Here are some examples:
"""

# Suffix: The actual query to answer
suffix = """
Question: {question}
[Answer]:
"""

# Create the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples_formatted,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["question"],
)

print(few_shot_prompt)

final_prompt = few_shot_prompt.format(question=question)
response = llm(final_prompt, max_new_tokens=2000)
print(response)

{'question': 'List all sports teams Craig Bryson, also known as Craig James Bryson, played for from 2010 to 2020.', 'answers': 'Kilmarnock F.C. (2010, 2011)\nScotland national football team (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)\nDerby County F.C. (2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)\nAberdeen F.C. (2019, 2020)'}
input_variables=['question'] examples=[{'question': 'List all sports teams Craig Bryson, also known as Craig James Bryson, played for from 2010 to 2020.', 'answers': 'Kilmarnock F.C. (2010, 2011)\nScotland national football team (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)\nDerby County F.C. (2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)\nAberdeen F.C. (2019, 2020)'}, {'question': 'List all sports teams Craig Gordon, also known as Craig Sinclair Gordon, played for from 2010 to 2020.', 'answers': 'Sunderland A.F.C. (2010, 2011, 2012)\nCeltic F.C. (2014, 2015, 2016, 2017, 2018, 2019, 2020)'},

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Birmingham City F.C. (2010, 2011) Birmingham City F.C. (2010
